In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="797b22135893422ab981597948963843",
                                               client_secret="80f412c2e1fa4a6999d52767d3d5fc7b",
                                               redirect_uri="http://127.0.0.1:7146/callback",
                                               scope="user-library-read"
                                               ))



In [ ]:

#Sample API call
taylor_uri = 'spotify:artist:06HL4z0CvFAxyc27GXpf02'
results = sp.artist_albums(taylor_uri, album_type='album')

albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY
THE TORTURED POETS DEPARTMENT
1989 (Taylor's Version) [Deluxe]
1989 (Taylor's Version)
Speak Now (Taylor's Version)
Midnights (The Til Dawn Edition)
Midnights (3am Edition)
Midnights
Red (Taylor's Version)
Fearless (Taylor's Version)
evermore (deluxe version)
evermore
folklore: the long pond studio sessions (from the Disney+ special) [deluxe edition]
folklore (deluxe version)
folklore
Lover
Taylor Swift Karaoke: reputation
reputation
reputation Stadium Tour Surprise Song Playlist
1989 (Deluxe Edition)
1989
Red (Deluxe Edition)
Red
Speak Now World Tour Live
Speak Now (Deluxe Edition)
Speak Now
Fearless Platinum Edition
Fearless
Live From Clear Channel Stripped 2008
Taylor Swift


In [ ]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
from IPython.display import display, HTML

#Sample JSON request deserialization
try:
    imageNames = []
    imageURIs = []
    request = sp.categories(limit = 50, offset = 49)
    for x in request['categories']['items']:
        imageNames.append(x['name'])
        for y in x['icons']:
            
            imageURIs.append(y['url'])

    html = ''.join([
    f'<div style="display:inline-block; text-align:center; margin:10px;">'
    f'<img src="{uri}" width="150"><br><span>{name}</span></div>'
    for name, uri in zip(imageNames, imageURIs)
    ])
    display(HTML(html))    
except SpotifyException as e:
    print(e)


HTTP Error for GET to https://api.spotify.com/v1/recommendations/available-genre-seeds with Params: {} returned 404 due to None


http status: 404, code: -1 - https://api.spotify.com/v1/recommendations/available-genre-seeds:
 None, reason: None


In [70]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
from IPython.display import display, HTML
import json

#Song tags:

query = "artist:taylor swift,album:1989"
request = sp.search(q = query, type = 'album', limit = 10, offset = 0)
#print(json.dumps(request, indent = 2))
albumIDs = []
albumImages = []

#Name (year), artist
albumText = []
for album in request['albums']['items']:
    albumIDs.append(album['id'])

    albumImages.append(album['images'][0]['url'])

    artists = []
    for artist in album['artists']:
        artists.append(artist['name'])
    albumText.append(f"{album['name']}({album['release_date'].split('-')[0]}) by {', '.join(artists)}")

html = ''.join([
    f'''
    <div style="display:inline-block; width:200px; text-align:center; margin:10px;">
        <img src="{uri}" width="200"><br>
        <span style="display:block; word-wrap:break-word; white-space:normal;">{name}</span>
    </div>
    '''
    for name, uri in zip(albumText, albumImages)
])

display(HTML(html))        

